# Exploring connectivity of the cities

In [5]:
import pandas as pd

## Loading flights

In [6]:
flights_df = pd.read_csv('data/flights.csv')
flights_df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
1,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
2,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
3,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
4,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


## Loading airports

In [8]:
airports_df = pd.read_csv('data/airports.csv')
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


# Making cities dataframe

In order to evaluate conntectivity of 2 cities we would like to count number of flights between those cities both ways.

For that we need a direction invariant, unique score, that we can assign to every city.

So the multiplication of those scores will give us unique score/identifier of a connection.

### Trick

Since every airport in our dataset has latittude and longitude, we can use their multiplication to define a score/identifier of a city. This multiplication should be a unique enoght city identifier for our task.




In [36]:
cities_df = airports_df[['IATA_CODE', 'CITY', 'LATITUDE', 'LONGITUDE']].copy()

cities_df['location_id'] = cities_df['LATITUDE'] * cities_df['LONGITUDE']

cities_df.head()

,IATA_CODE,CITY,LATITUDE,LONGITUDE,location_id
0,ABE,Allentown,40.65236,-75.44040,-3066.830299
1,ABI,Abilene,32.41132,-99.68190,-3230.821959
2,ABQ,Albuquerque,35.04022,-106.60919,-3735.609472
3,ABR,Aberdeen,45.44906,-98.42183,-4473.179657
4,ABY,Albany,31.53552,-84.19447,-2655.116393


## Preparing flights for our analysis

In [37]:
df = flights_df[['origin', 'dest']].copy()

df = pd.merge(df, cities_df[['IATA_CODE', 'CITY', 'location_id']], how='left', left_on='origin', right_on='IATA_CODE')
df = df.rename({'CITY': 'origin_city', 'location_id': 'origin_location_id'}, axis='columns')
df = df.drop('IATA_CODE', axis='columns')

df = pd.merge(df, cities_df[['IATA_CODE', 'CITY', 'location_id']], how='left', left_on='dest', right_on='IATA_CODE')
df = df.rename({'CITY': 'dest_city', 'location_id': 'dest_location_id'}, axis='columns')
df = df.drop('IATA_CODE', axis='columns')

df['connection_id'] = df['origin_location_id'] * df['dest_location_id']

scored_connections_df = df

df.head()

,origin,dest,origin_city,origin_location_id,dest_city,dest_location_id,connection_id
0,EWR,IAH,Newark,-3018.108197,Houston,-2858.329615,8.626748e+06
1,LGA,IAH,New York,-3012.321147,Houston,-2858.329615,8.610207e+06
2,JFK,MIA,New York,-2998.357270,Miami,-2070.954487,6.209461e+06
3,JFK,BQN,New York,-2998.357270,Aguadilla,-1241.549595,3.722609e+06
4,LGA,ATL,New York,-3012.321147,Atlanta,-2840.159409,8.555472e+06


## Now, when every connection between 2 cities has it's unique score independently from flight direction we can count these connections.

In [40]:

flights_per_connection_df = scored_connections_df.groupby(['connection_id'])

df = flights_per_connection_df['origin_city'].agg(wtf=('city_1', {lambda x: x.iloc[0]}))
df['city_2'] = flights_per_connection_df['dest_city'].agg(wtf=('city_2', {lambda x: x.iloc[0]}))
df['flights_count'] = flights_per_connection_df.count()[['origin']]

df = df.sort_values(by=['flights_count'], ascending=False)

df.head(30)

,city_1,city_2,flights_count
,<lambda>,,
connection_id,,,
1.205061e+07,New York,Los Angeles,11262
8.555472e+06,New York,Atlanta,10263
1.111605e+07,New York,Chicago,8857
1.380329e+07,New York,San Francisco,8204
8.586116e+06,New York,Charlotte,6168
1.113740e+07,Newark,Chicago,6100
9.019323e+06,New York,Boston,5898
6.238380e+06,New York,Miami,5781


# Other approach

In [16]:


import numpy as np
df = flights_df[['origin', 'dest']].copy()

def combine_origin_and_destination(origin, dest):
    origin_and_dest = [origin, dest]
    origin_and_dest.sort()
    return '-'.join(origin_and_dest)

df['connection_id'] = [combine_origin_and_destination(x, y) for x, y in zip(df['origin'].map(str), df['dest'].map(str))]

scored_connections_df = df

df.head(20)

,origin,dest,connection_id
0,EWR,IAH,EWR-IAH
1,LGA,IAH,IAH-LGA
2,JFK,MIA,JFK-MIA
3,JFK,BQN,BQN-JFK
4,LGA,ATL,ATL-LGA
5,EWR,ORD,EWR-ORD
6,EWR,FLL,EWR-FLL
7,LGA,IAD,IAD-LGA
8,JFK,MCO,JFK-MCO
9,LGA,ORD,LGA-ORD


In [34]:
flights_per_connection_df = scored_connections_df.groupby('connection_id').size().sort_index().reset_index()

#flights_per_connection_df = flights_per_connection_df.rename({'origin': 'flights_count', axis='columns')
#print(flights_per_connection_df)
flights_per_connection_df = flights_per_connection_df.sort_values(by=[0], ascending=False)

flights_per_connection_df.head()


connection_id     0
0         ABQ-JFK   254
1         ACK-JFK   265
2         ALB-EWR   439
3         ANC-EWR     8
4         ATL-EWR  5022
..            ...   ...
219       LGA-SYR   293
220       LGA-TPA  2145
221       LGA-TVC    77
222       LGA-TYS   308
223       LGA-XNA   745

[224 rows x 2 columns]


KeyError: 1